# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for x1,y1,x2,y2 in lines:
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def cluster_lines(lines):
    lines_m = np.concatenate(lines[:]) #array of lines as a matrix
    #finds line parameters y=ax+b
    aa =[slope(x) for x in lines_m]
    bb =[intercept(x) for x in lines_m]
    #cleans the data, removes NaN and inf (better to have in polar coordinates)
    a=[]
    b=[]
    for i in range(np.size(aa)):
        if (~np.isnan(aa[i])) & (~np.isinf(aa[i])) & (~np.isnan(bb[i])) & (~np.isinf(bb[i])):
            a.append(aa[i])
            b.append(bb[i])
    #finds the two clusters of lines with most frequent pair slope/intercepts
    (H,ex,ey)=np.histogram2d(a,b)
    cols=np.argsort(H.flatten())[[-1,-2]]%10
    rows=np.floor_divide(np.argsort(H.flatten())[[-1,-2]],10)
    #indeces for each set of lines
    index1=(a>=ex[rows[0]]) & (a<=ex[rows[0]+1])& (b>=ey[cols[0]]) & (b<=ey[cols[0]+1])
    index2=(a>=ex[rows[1]]) & (a<=ex[rows[1]+1])& (b>=ey[cols[1]]) & (b<=ey[cols[1]+1])
    lines1=lines_m[index1,:]
    lines2=lines_m[index2,:]
    #check which are the left and right lines
    if np.mean([slope(x) for x in lines1])<0:
        lines_l=lines1
        lines_r=lines2
    else:
        lines_l=lines2
        lines_r=lines1
    return (lines_l,lines_r)

def merge_lines(lines_l,lines_r):
    #estimates an average line
    a_l=np.mean([slope(x) for x in lines_l])
    a_r=np.mean([slope(x) for x in lines_r])
    b_l=np.mean([intercept(x) for x in lines_l])
    b_r=np.mean([intercept(x) for x in lines_r])   
    #finds the starting point
    ys=540 #bottom
    xs_l=(ys-b_l)/a_l
    xs_r=(ys-b_r)/a_r
    #finds the final point   
    yf=np.min(np.append(lines_l[:,[1,3]],lines_r[:,[1,3]]))#top of detected lines
    xf_l=(yf-b_l)/a_l
    xf_r=(yf-b_r)/a_r    
    #merged lines
    line_l_m = [int(xs_l),int(ys),int(xf_l),int(yf)]
    line_r_m = [int(xs_r),int(ys),int(xf_r),int(yf)]
    return ([line_l_m],[line_r_m])

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns a set of lines.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def slope(line):
    #calculates the angle (in deg) between two line segments
    (x1,y1,x2,y2)=line
    return float(y2-y1)/(x2-x1)

def intercept(line):
    (x1,y1,x2,y2)=line
    return (x2*y1-x1*y2)/(x2-x1)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
im_files=os.listdir("test_images/")
for im_file in im_files:
    #reading in an image
    img = mpimg.imread('test_images/'+im_file)
    #printing out some stats
    print('This image is:', im_file, 'with dimesions:', img.shape)
    img_g = grayscale(img)
    #applies a Gaussian filter
    img_g_b = gaussian_blur(img_g,5)
    #applies a Canny filter
    img_g_b_c = canny(img_g_b,50,150)
    #selects region of interest
    #vertices = np.array([[(50,540),(380, 200), (700, 200), (960,540)]])
    imshape=img.shape
    vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    img_g_b_c_i = region_of_interest(img_g_b_c, vertices)
    #hough transform filter
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments   
    lines_h = hough_lines(img_g_b_c_i, rho, theta, threshold, min_line_len, max_line_gap)
    img_g_b_c_i_h = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #finds two clusters of lines
    (lines_l,lines_r)=cluster_lines(lines_h)
    #merge left and right sets
    (line_l_m,line_r_m)=merge_lines(lines_l,lines_r)
    #draws lines
    draw_lines(img_g_b_c_i_h,line_l_m)
    draw_lines(img_g_b_c_i_h,line_r_m)

    #overlay images
    img_l = weighted_img(img_g_b_c_i_h, img)
    #plot each stage of the algorithm, except blurring
    
    
    plt.subplots(figsize=(20, 10))
    plt.subplot(161)
    plt.imshow(img)
    plt.subplot(162)        
    plt.imshow(img_g,cmap='gray')
    plt.subplot(163)
    plt.imshow(img_g_b_c,cmap='gray')
    plt.subplot(164)
    plt.imshow(img_g_b_c_i,cmap='gray')
    plt.subplot(165)
    plt.imshow(img_g_b_c_i_h)
    plt.subplot(166)
    plt.imshow(img_l)


run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(img):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    img_g = grayscale(img)
    #applies a Gaussian filter
    img_g_b = gaussian_blur(img_g,5)
    #applies a Canny filter
    img_g_b_c = canny(img_g_b,50,150)
    #selects region of interest
    #vertices = np.array([[(50,540),(380, 200), (700, 200), (960,540)]])
    imshape=img.shape
    vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    img_g_b_c_i = region_of_interest(img_g_b_c, vertices)
    #hough transform filter
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments   
    lines_h = hough_lines(img_g_b_c_i, rho, theta, threshold, min_line_len, max_line_gap)
    img_g_b_c_i_h = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #finds two clusters of lines
    (lines_l,lines_r)=cluster_lines(lines_h)
    #merges the two lines
    (line_l_m,line_r_m)=merge_lines(lines_l,lines_r)
    #draws lines
    draw_lines(img_g_b_c_i_h,line_l_m,[255,0,0],5)
    draw_lines(img_g_b_c_i_h,line_r_m,[255,0,0],5)
    #overlay images
    img_l = weighted_img(img_g_b_c_i_h, img)
    result=img_l
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


Some possible improvements to this algorithm
1) Make it less dependent on absolute pixel positioning: the tuning variables for the region of interest and Hough transform are based on absolute pixel values. This will depend considerably on the setup, e.g. the camera's resolution and its positioning. It would be better to consider how the actual environmental setup (e.g. length of lanes) will translate into pixels according to a camera model. This would lead to a more generally applicable algorithm.
2) Lane tracking: in order to better track the lanes and avoid jumpy behavior of lanes, it would be better to track these features using a tracking filter, that takes into account how fast the car is moving and how fast a change in lane direction can be expected between frames. This can also be used to reduce the amount of clutter data by considering data association.
3) Geometrical features: while a straight line model is simple, it does not represent the lanes during turns. Considering a lane model that admits curves could lead to better performance. In fact, it is possible to use more geometric information to improve the algorithm, e.g. that the lanes are symmetric or that they are located within the same height (depending on the camera location relative to the car). Estimating the entire area between the lanes instead of the lanes themselves could possibly be better since more pixels (information) can be considered.
4) Road segmentation: in order to remove the effects of the background, it would be useful to detect the limits of the road, leading to a more appropriate region of interest. This can possibly be done considering features of the road such as color and homogeneity. 
5) Car detection: to avoid creating lines over the cars, it would be good to remove the pixels where cars have been detected.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))